In [1]:
import os
import copy
import pickle
import numpy as np

from simtk.openmm import unit
from tqdm import tqdm_notebook

from scipy.stats import pearsonr


In [38]:
def get_ci_for_pcc(n_replicas, out_dir, sub_dir, phase, angle=None, res=None, residue_pair=None, is_water=False, 
                   position_type='old', is_checkpoint_interval_10=False, sin_transform=False, cos_transform=False):
    """
    Compute confidence interval for each Pearson correlation coefficient (PCC)

    Parameters
    ----------
    n_replicas : int 
        number of replicas used in the simulation
    out_dir : str
        path to directory containing data to analyze
    sub_dir : str
        path to sub-directory containing data to analyze
    phase : str
        phase (e.g., 'complex')
    angle : str, default None
        dihedral angle name (i.e., 'phi', 'psi', 'chi1', 'chi2', 'chi3', 'chi4')
    res : str, default None
        chain name and residue id (format: {barnase|barstar}-{residue id})
        example: 'barstar-42'
    residue_pair : str, default None
        residue pair name (format: {barnase|barstar}-{residue id} | {barnase|barstar}-{residue id})
        example: 'barstar-42 | barstar-76 new'
    is_water : boolean, default False
        whether the dof to analyze is neighboring waters
    position_type : str, default 'old'
        the type of positions (i.e., 'old' or 'new')
    checkpoint_interval_10 : boolean, default False
        whether the simulation was run with checkpoint interval of 10 
    sin_transform : boolean, default False
        whether to apply a sine transformation the dihedral angle type
    cos_transform : boolean, default False
        whether to apply a cosine transformation the dihedral angle type
        
    Returns
    -------
    pcc : float
        Pearson correlation coefficient (PCC)
    np.percentile(pcc_bootstraps, 2.5) : float
        lower bound of 95% CI for PCC
    np.percentile(pcc_bootstraps, 97.5) : float
        upper bound of 95% CI for PCC
    """
    
    # Load du_dlambda
    du_dlambda_all = []
    for replica in range(n_replicas):
        with open(os.path.join(out_dir, f"{sub_dir}_{phase}_du_dlambda_replica{replica}.npy"), "rb") as f:
            du_dlambda = np.load(f)
        if is_checkpoint_interval_10:
            du_dlambda = du_dlambda[::10]
        du_dlambda_all.append(du_dlambda[:501])
    du_dlambda_all = [val * unit.kilojoules_per_mole.conversion_factor_to(unit.kilocalories_per_mole) for val in du_dlambda_all]

    if angle: # Get angle values
        y = []
        for replica in range(n_replicas):
            with open(os.path.join(out_dir, f"dataframe_dihedral_angles_per_replica{replica}.pickle"), "rb") as f:
                df = pickle.load(f)
            data = df[df["residue"] == res][f"{angle}_{position_type}"].values
            if is_checkpoint_interval_10: # Truncate, if necessary
                data = data[::10]
            y.append(data[:501])

    elif residue_pair: # Get residue distances
        y = []
        for replica in range(n_replicas):
            with open(os.path.join(out_dir, f"dataframe_residue_distances_replica{replica}.pickle"), "rb") as f:
                df = pickle.load(f)
            data = df[residue_pair].values
            if is_checkpoint_interval_10: # Truncate, if necessary
                data = data[::10]
            y.append(data[:501])

    elif is_water: # Get water values
        with open(os.path.join(out_dir, f"dataframe_waters_per_replica.pickle"), "rb") as f:
            df = pickle.load(f)
        y = []
        for replica in range(n_replicas):
            data = df[f"waters nearby mutating res (radius = 0.5 nm) {position_type}"][df["replica"] == replica].values
            if is_checkpoint_interval_10: # Truncate, if necessary
                data = data[::10]
            y.append(data[:501])
    
    # Sample PCC
    du_dlambda_all_concat = np.concatenate(du_dlambda_all)
    y_concat = np.concatenate(y)
    if sin_transform:
        y_concat = np.sin(y_concat)
    elif cos_transform:
        y_concat = np.cos(y_concat)
    pcc = pearsonr(du_dlambda_all_concat, y_concat).statistic
    
    # Bootstrap and compute PCCs
    n_bootstraps = 200
    pcc_bootstraps = []
    du_dlambda_all = np.array(du_dlambda_all)
    y = np.array(y)
    for _ in range(n_bootstraps):
        # Subsample replica indices
        num_samples = du_dlambda_all.shape[0]
        subsample_indices = np.random.choice(range(num_samples), num_samples)
        du_dlambda_bootstrapped = np.concatenate(du_dlambda_all[subsample_indices])
        y_bootstrapped = np.concatenate(y[subsample_indices])
        
        # Compute PCC
        if sin_transform:
            y_bootstrapped = np.sin(y_bootstrapped)
        elif cos_transform:
            y_bootstrapped = np.cos(y_bootstrapped)
        pcc_bootstrapped = pearsonr(du_dlambda_bootstrapped, y_bootstrapped).statistic
        pcc_bootstraps.append(pcc_bootstrapped)

    return pcc, np.percentile(pcc_bootstraps, 2.5), np.percentile(pcc_bootstraps, 97.5)

In [41]:
phase = 'complex'
for sub_dir in tqdm_notebook(range(28)):
    print(f"sub_dir: {sub_dir}")
    
    if sub_dir == 4:
        main_dir = 45
        sub_dir = 11
        replicate = 0
        n_replicas = 24
        is_checkpoint_interval_10 = False

    elif sub_dir == 12:
        main_dir = 45
        sub_dir = 10
        replicate = 1
        n_replicas = 36
        is_checkpoint_interval_10 = True

    elif sub_dir == 17:
        main_dir = 45
        sub_dir = 9
        replicate = 1
        n_replicas = 24
        is_checkpoint_interval_10 = True

    else:
        main_dir = 47
        replicate = 1
        is_checkpoint_interval_10 = False
        n_replicas = 24 if sub_dir in [0, 1, 2, 3, 14, 15, 16, 18] else 36
    
    # Load correlation data -- dict of key: name of degree of freedom, value: dict containing 'pearsonr' and 'tranform_type' as keys
    out_dir = f"/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/{main_dir}/{sub_dir}/replicate_{replicate}/"
    with open(os.path.join(out_dir, f"correlation_data_replicas_all_50ns.pickle"), "rb") as f:
        correlation_data = pickle.load(f)

    pccs = []
    cis = []
    for i, (k, v) in enumerate(correlation_data.items()):

        # Dihedral angles
        if i in [0, 1]: 
            res, angle, position_type = k.split(" ")
            residue_pair = None
            is_water = False
            if v['transform_type'] == 'sin':
                sin_transform = True
                cos_transform = False
            else:
                sin_transform = False
                cos_transform = True

        # Residue distances
        elif i in [2, 3]:
            angle = None
            res = None
            residue_pair = k
            is_water = False
            position_type = None
            sin_transform = False
            cos_transform = False

        # Waters
        elif i == 4:
            angle = None
            res = None
            residue_pair = None
            is_water = True
            position_type = k
            sin_transform = False
            cos_transform = False
        
        # Compute pcc and ci
        pcc, ci_low, ci_high = get_ci_for_pcc(n_replicas, out_dir, sub_dir, phase, 
                                               angle=angle, 
                                               res=res, 
                                               residue_pair=residue_pair, 
                                               is_water=is_water, 
                                               position_type=position_type, 
                                               is_checkpoint_interval_10=checkpoint_interval_10,
                                               sin_transform=sin_transform,
                                               cos_transform=cos_transform)

        pccs.append(pcc)
        cis.append((ci_low, ci_high))
    
    # Save
    out_dir = f"/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/{main_dir}/{sub_dir}/replicate_{replicate}/"
    with open(os.path.join(out_dir, f"correlation_data_replicas_all_with_cis_50ns.pickle"), "wb") as f:
        pickle.dump(pccs + cis, f)


/scratch/lsftmp/14514146.tmpdir/ipykernel_10183/1369809967.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sub_dir in tqdm_notebook(range(28)):


  0%|          | 0/28 [00:00<?, ?it/s]

sub_dir: 0
sub_dir: 1
sub_dir: 2
sub_dir: 3
sub_dir: 4
sub_dir: 5
sub_dir: 6
sub_dir: 7
sub_dir: 8
sub_dir: 9
sub_dir: 10
sub_dir: 11
sub_dir: 12
sub_dir: 13
sub_dir: 14
sub_dir: 15
sub_dir: 16
sub_dir: 17
sub_dir: 18
sub_dir: 19
sub_dir: 20
sub_dir: 21
sub_dir: 22
sub_dir: 23
sub_dir: 24
sub_dir: 25
sub_dir: 26
sub_dir: 27
